<a href="https://colab.research.google.com/github/ropitz/soils/blob/main/soil_concept_network_viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Make an Interactive Network Visualization with Bokeh

## Dataset

## Import Libraries

In [34]:
import pandas as pd
import networkx
import matplotlib.pyplot as plt
import numpy as np

## Install and Import Bokeh

In [35]:
#!pip install bokeh

In [36]:
from bokeh.io import output_notebook, show, save

To view interactive Bokeh visualizations in a Jupyter notebook, you need to run this cell:

In [37]:
output_notebook()

## Create Network From Pandas DataFrame

In [38]:
got_df = pd.read_csv('https://raw.githubusercontent.com/ropitz/soils/main/summary_concept_maps.csv')

In [39]:
got_df

,technica,concept
0,Acidity (pH),Capturing Critical Zone processes
1,Acidity (pH),Erosion
2,Acidity (pH),Evidencing ecosystem processes
3,Acidity (pH),Evidencing ecosystem processes
4,Acidity (pH),Evidencing human-ecosystem interactions
...,...,...
189,Texture,Supporting a living ecosystem
190,Texture,Sustaining animal health
191,Texture,Sustaining human health
192,Texture,Sustaining plant health


Then we make a network with `networkx.from_pandas_edgelist()`:

In [40]:
G = networkx.from_pandas_edgelist(got_df, 'technica', 'concept')

## Basic Network 

The code below shows how to make a basic network viz that includes Hover Tooltips (a text box that will display when a user hovers over nodes) as well as Zoom and Pan/Drag functionality.

For more details about visualizing network graphs with Bokeh, see [the documentation](https://docs.bokeh.org/en/latest/docs/user_guide/graph.html?highlight=networks).

In [41]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx

In [42]:
#Choose a title!
title = 'soil health and heritage'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [("concept", "@index")]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object with spring layout
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))

#Set node size and color
network_graph.node_renderer.glyph = Circle(size=15, fill_color='skyblue')

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

#Add network graph to the plot
plot.renderers.append(network_graph)

show(plot)
#save(plot, filename=f"{title}.html")

## Network with Nodes Sized and Colored By Attribute (Degree)

The code below shows how to size and color nodes by degree.

**Include Bokeh color palettes**

In [43]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap

**Calculate degree for each node and add as node attribute**

In [44]:
degrees = dict(networkx.degree(G))
networkx.set_node_attributes(G, name='degree', values=degrees)

**Slightly adjust degree so that the nodes with very small degrees are still visible**

In [45]:
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in networkx.degree(G)])
networkx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)

In [46]:
#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'adjusted_node_size'

#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Blues8

#Choose a title!
title = 'soil health and heritage'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
       ("concept", "@index"),
        ("Degree", "@degree")
]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html\
network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color = min(network_graph.node_renderer.data_source.data[color_by_this_attribute])
maximum_value_color = max(network_graph.node_renderer.data_source.data[color_by_this_attribute])
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

plot.renderers.append(network_graph)

show(plot)
#save(plot, filename=f"{title}.html")

## Network with Nodes Colored By Attribute (Community)

The code below shows how to size and color nodes by modularity class.

**Include community module**

In [47]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
from networkx.algorithms import community

**Calculate degree for each node and add as node attribute**

In [48]:
degrees = dict(networkx.degree(G))
networkx.set_node_attributes(G, name='degree', values=degrees)

**Slightly adjust degree so that the nodes with very small degrees are still visible**

In [49]:
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in networkx.degree(G)])
networkx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)

**Calculate communities**

In [50]:
communities = community.greedy_modularity_communities(G)

**Add modularity class and color as attributes to network graph**

In [51]:
# Create empty dictionaries
modularity_class = {}
modularity_color = {}
#Loop through each community in the network
for community_number, community in enumerate(communities):
    #For each member of the community, add their community number and a distinct color
    for name in community: 
        modularity_class[name] = community_number
        modularity_color[name] = Spectral8[community_number]

In [52]:
# Add modularity class and color as attributes from the network above
networkx.set_node_attributes(G, modularity_class, 'modularity_class')
networkx.set_node_attributes(G, modularity_color, 'modularity_color')

In [53]:
#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'modularity_color'
#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Blues8
#Choose a title!
title = 'soil health and heritage'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
       ("concept", "@index"),
        ("Degree", "@degree"),
         ("Modularity Class", "@modularity_class"),
        ("Modularity Color", "$color[swatch]:modularity_color"),
]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset, tap", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as category from attribute)
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=color_by_this_attribute)

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

plot.renderers.append(network_graph)

show(plot)
#save(plot, filename=f"{title}.html")

## Network with Responsive Highlighting

The code below shows how to create responsive highlighting when a user hovers over nodes or edges, which you can read more about in [Bokeh's NetworkX Integration documentation](https://docs.bokeh.org/en/latest/docs/user_guide/graph.html?highlight=networks#interaction-policies). 

**Include  EdgesAndLinkedNodes, NodesAndLinkedEdges**

In [54]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine, EdgesAndLinkedNodes, NodesAndLinkedEdges
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
from networkx.algorithms import community

**Calculate degree for each node and add as node attribute**

In [55]:
degrees = dict(networkx.degree(G))
networkx.set_node_attributes(G, name='degree', values=degrees)

**Slightly adjust degree so that the nodes with very small degrees are still visible**

In [56]:
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in networkx.degree(G)])
networkx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)

**Calculate communities**

In [57]:
communities = community.greedy_modularity_communities(G)

**Add modularity class and color as attributes to network graph**

In [58]:
# Create empty dictionaries
modularity_class = {}
modularity_color = {}
#Loop through each community in the network
for community_number, community in enumerate(communities):
    #For each member of the community, add their community number and a distinct color
    for name in community: 
        modularity_class[name] = community_number
        modularity_color[name] = Spectral8[community_number]

In [59]:
# Add modularity class and color as attributes from the network above
networkx.set_node_attributes(G, modularity_class, 'modularity_class')
networkx.set_node_attributes(G, modularity_color, 'modularity_color')

In [60]:
from bokeh.models import EdgesAndLinkedNodes, NodesAndLinkedEdges

#Choose colors for node and edge highlighting
node_highlight_color = 'white'
edge_highlight_color = 'black'

#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'modularity_color'

#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Blues8

#Choose a title!
title = 'soil health and heritage'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
       ("concept", "@index"),
        ("Degree", "@degree"),
         ("Modularity Class", "@modularity_class"),
        ("Modularity Color", "$color[swatch]:modularity_color"),
]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as category from attribute)
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=color_by_this_attribute)
#Set node highlight colors
network_graph.node_renderer.hover_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)
network_graph.node_renderer.selection_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)
#Set edge highlight colors
network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)

    #Highlight nodes and edges
network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()

plot.renderers.append(network_graph)

show(plot)
#save(plot, filename=f"{title}.html")

## Network with Labels

The code below shows how to create node labels, which you can read more about in [Bokeh's label documentation](https://docs.bokeh.org/en/latest/docs/user_guide/annotations.html#labels).

**Include  LabelSet**

In [61]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine, EdgesAndLinkedNodes, NodesAndLinkedEdges, LabelSet
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
from networkx.algorithms import community

**Calculate degree for each node and add as node attribute**

In [62]:
degrees = dict(networkx.degree(G))
networkx.set_node_attributes(G, name='degree', values=degrees)

**Slightly adjust degree so that the nodes with very small degrees are still visible**

In [63]:
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in networkx.degree(G)])
networkx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)

**Calculate communities**

In [64]:
communities = community.greedy_modularity_communities(G)

**Add modularity class and color as attributes to network graph**

In [65]:
# Create empty dictionaries
modularity_class = {}
modularity_color = {}
#Loop through each community in the network
for community_number, community in enumerate(communities):
    #For each member of the community, add their community number and a distinct color
    for name in community: 
        modularity_class[name] = community_number
        modularity_color[name] = Spectral8[community_number]

In [66]:
#Choose colors for node and edge highlighting
node_highlight_color = 'white'
edge_highlight_color = 'black'

#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'modularity_color'

#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Blues8

#Choose a title!
title = 'soil health and heritage'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
       ("concept", "@index"),
        ("Degree", "@degree"),
         ("Modularity Class", "@modularity_class"),
        ("Modularity Color", "$color[swatch]:modularity_color"),
]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as category from attribute)
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=color_by_this_attribute)
#Set node highlight colors
network_graph.node_renderer.hover_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)
network_graph.node_renderer.selection_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.3, line_width=1)
#Set edge highlight colors
network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)

#Highlight nodes and edges
network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()

plot.renderers.append(network_graph)

#Add Labels
x, y = zip(*network_graph.layout_provider.graph_layout.values())
node_labels = list(G.nodes())
source = ColumnDataSource({'x': x, 'y': y, 'name': [node_labels[i] for i in range(len(x))]})
labels = LabelSet(x='x', y='y', text='name', source=source, background_fill_color='white', text_font_size='10px', background_fill_alpha=.7)
plot.renderers.append(labels)

show(plot)
#save(plot, filename=f"{title}.html")

In [69]:
from operator import itemgetter

In [72]:
#Betweenness Centrality
betweenness_dict = networkx.betweenness_centrality(G) #Create a dictionary and run the measurement
networkx.set_node_attributes(G, betweenness_dict, 'Betweenness') #Add the betweeness centrality as an attribute to the nodes

#We'll now sort the cities by their betweenness centrality and print a "Top 5" list
sorted_betweenness = sorted(betweenness_dict.items(), key=itemgetter(1), reverse=True)

print("Top 5 Concepts by Betweenness Centrality:")
for b in sorted_betweenness[:5]:
    print(b)


Top 5 Concepts by Betweenness Centrality:
('Moisture', 0.12978910970052002)
('Supporting a living ecosystem', 0.08855583454258253)
('Carbon Content', 0.08796207142300787)
('Organic Matter Content', 0.08796207142300787)
('Sustaining plant health', 0.08035117215411909)


In [71]:
#Let's create a dictionary to add the degree and run "G.degree()" in it.
degree_dict = dict(G.degree(G.nodes()))
networkx.set_node_attributes(G, degree_dict, 'degree') #We add the degree as an attribute to the nodes

#Now we sort the nodes by degree, making sure to put the higher degrees first with "reverse=True"
sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)
print("Concepts by Degree:", sorted_degree) #Remember to print and name your list!

Concepts by Degree: [('Moisture', 15), ('Carbon Content', 14), ('Organic Matter Content', 14), ('Sustaining plant health', 13), ('Storing Carbon', 12), ('Supporting a living ecosystem', 12), ('Archiving evidence of past human activities', 11), ('Porosity ', 11), ('Erosion', 10), ('Nutrient Content ', 10), ('Acidity (pH)', 9), ('Evidencing human-ecosystem interactions', 9), ('Archiving evidence of past ecosystem processes', 9), ('Structure', 9), ('Texture', 9), ('Capturing Critical Zone processes', 8), ('Evidencing ecosystem processes', 8), ('Sustaining animal health', 7), ('Sustaining human health', 7), ('Compaction', 7), ('Regulating Climate', 6), ('Colour', 6), ('Restoration', 5), ('Bulk Density', 5), ('Temperature', 5), ('Inorganic inclusions', 4), ('Forming rural landscapes', 3), ('Modification', 2), ('Organisms present', 2), ('Resistivity ', 2)]


In [74]:
#Density Analysis
density = networkx.density(G) #This calculates the density
print("Concept Network Density:", density) #Let's print it and name it so that we do not forget

#Diameter Analysis
diameter = networkx.diameter(G)
print("Concept Network Diameter", diameter)

Concept Network Density: 0.28045977011494255
Concept Network Diameter 4
